# Načtení C shared knihovny (`.so`) pomocí Ctypes (uvnitř NumPy)

Moduly Ctypes a Numpy lze použít pro snadnou integraci mezi jazyky Python a C.

- `Ctypes` je modul Pythonu, který poskytuje kompatibilní datové typy pro C a Python, a umožňuje volat funkce z dynamicky sdílených knihoven napsaných v jazyce C.
- `Numpy` obsahuje integraci `Ctypes` - lze tedy použít čistě Ctypes, případně Ctypes podmodul v Numpy


Pro nalinkování předchozí funkce `matrix_vector_multiply` z C knihovny `libmatrix_vector_multiply.so` je třeba:


1. Načítání dynamicky sdílené knihovny: Kód načítá dynamicky sdílenou knihovnu (také známou jako "shared library" nebo ".so" soubor), která obsahuje funkce a datové struktury napsané v jazyce C. Toto se provádí pomocí `ctypes.CDLL` nebo `np.ctypeslib.load_library`.
2. Přistoupení k oběktu reprezentujícímu C funkci (používá se `.` syntaxe, jako u modulů)
3. Mapování C datových typů na Python datové typy: Kód mapuje C datové typy na jejich odpovídající Python datové typy pomocí ctypes modulu.  Definuje se atribut `argtypes` pro každou C funkci, která se volá, a `restype` pro funkce, které vrací hodnotu.


Více informací o Ctypes naleznete v [dokumentaci](https://docs.python.org/3/library/ctypes.html), nebo v NumPy [dokumentaci](https://numpy.org/doc/stable/reference/routines.ctypeslib.html).

In [ ]:
import numpy as np

# Load the shared library
lib_matrix_vector_multiply = np.ctypeslib.load_library(libname='libmatrix_vector_multiply.so', loader_path='.')

# Define the matrix_vector_multiply function from the shared library
matrix_vector_multiply = lib_matrix_vector_multiply.matrix_vector_multiply
matrix_vector_multiply.argtypes = [np.ctypeslib.ndpointer(dtype=np.float32),
                                   np.ctypeslib.ndpointer(dtype=np.float32),
                                   np.ctypeslib.ndpointer(dtype=np.float32),
                                   np.ctypeslib.c_intp,
                                   np.ctypeslib.c_intp]

matrix = np.array([[1, 2, 3],
                    [4, 5, 6]], dtype=np.float32)

vector = np.array([1, 2, 3], dtype=np.float32)
result = np.zeros((2), dtype=np.float32)

matrix_vector_multiply(matrix, vector, result, matrix.shape[0], matrix.shape[1])

print("Result vector:")
print(result)


In [ ]:
matrix@vector

Totéž přímo pomocí Ctypes:

In [ ]:
import ctypes
import numpy as np

# Načtěte sdílenou knihovnu
lib_matrix_vector_multiply = ctypes.CDLL('./libmatrix_vector_multiply.so')

# Definujte funkci matrix_vector_multiply ze sdílené knihovny
matrix_vector_multiply = lib_matrix_vector_multiply.matrix_vector_multiply
matrix_vector_multiply.argtypes = [ctypes.POINTER(ctypes.c_float),
                                   ctypes.POINTER(ctypes.c_float),
                                   ctypes.POINTER(ctypes.c_float),
                                   ctypes.c_int,
                                   ctypes.c_int]

matrix = np.array([[1, 2, 3],
                    [4, 5, 6]], dtype=np.float32)

vector = np.array([1, 2, 3], dtype=np.float32)
result = np.zeros((2), dtype=np.float32)

# Konverze numpy polí na C pole
matrix_ptr = matrix.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
vector_ptr = vector.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
result_ptr = result.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

matrix_vector_multiply(matrix_ptr, vector_ptr, result_ptr, matrix.shape[0], matrix.shape[1])

print("Result vector:")
print(result)


### Takto to dělá CFFI

Jednou z výhod CFFI je možnost automatického načtení typů z hlavičnového souboru.

In [ ]:
%%writefile matrix_vector_multiply.h

void matrix_vector_multiply(float *matrix, float *vector, float *result, int rows, int cols);

In [ ]:
#!pip install cffi

In [ ]:
import numpy as np
from cffi import FFI

ffi = FFI()

# Načtěte obsah hlavičkového souboru
with open('matrix_vector_multiply.h', 'r') as f:
    header = f.read()

# Zadejte hlavičkový soubor s deklarací funkce
ffi.cdef(header)

# Načtěte sdílenou knihovnu
lib_matrix_vector_multiply = ffi.dlopen('./libmatrix_vector_multiply.so')

matrix = np.array([[1, 2, 3],
                   [4, 5, 6]], dtype=np.float32)

vector = np.array([1, 2, 3], dtype=np.float32)
result = np.zeros((2), dtype=np.float32)

# Konverze numpy polí na C pole
matrix_ptr = ffi.cast("float *", matrix.ctypes.data)
vector_ptr = ffi.cast("float *", vector.ctypes.data)
result_ptr = ffi.cast("float *", result.ctypes.data)

lib_matrix_vector_multiply.matrix_vector_multiply(matrix_ptr, vector_ptr, result_ptr, matrix.shape[0], matrix.shape[1])

print("Result vector:")
print(result)

In [ ]:
matrix@vector